In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl  # noqa: F401
    import dolfinx  # noqa: F401
else:
    try:
        import ufl
        import dolfinx
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenicsx-install.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        import ufl  # noqa: F401
        import dolfinx  # noqa: F401

In [ ]:
try:
    import google.colab
except ImportError:
    pass
else:
    google.colab.output.enable_custom_widget_manager()
try:
    import pyvista
except ImportError:
    !pip3 install --upgrade pyvista itkwidgets
    import pyvista  # noqa: F401

In [ ]:
# Download data files
!mkdir -p data
![ -f data/circle.h5 ] || wget https://github.com/multiphenics/multiphenicsx/raw/main/tutorials/03_lagrange_multipliers/data/circle.h5 -O data/circle.h5
![ -f data/circle.xdmf ] || wget https://github.com/multiphenics/multiphenicsx/raw/main/tutorials/03_lagrange_multipliers/data/circle.xdmf -O data/circle.xdmf

In [ ]:
import numpy as np
from mpi4py import MPI
from dolfinx.fem import Function, FunctionSpace, VectorFunctionSpace
from dolfinx.io import XDMFFile
from dolfinx.mesh import GhostMode
from dolfinx.plot import create_vtk_topology

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/circle.xdmf", "r") as infile:
    mesh = infile.read_mesh(GhostMode.none)
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

In [ ]:
V_scalar = FunctionSpace(mesh, ("CG", 1))

In [ ]:
u_scalar = Function(V_scalar)
u_scalar.interpolate(lambda x: 9 - x[0]**2 - x[1]**2)

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_scalar_field_plot(mesh, u_scalar, "scalar")

In [ ]:
V_vector = VectorFunctionSpace(mesh, ("CG", 1))

In [ ]:
u_vector = Function(V_vector)
u_vector.sub(0).interpolate(lambda x: 2 * (9 - x[0]**2 - x[1]**2))
u_vector.sub(1).interpolate(lambda x: 3 * (9 - x[0]**2 - x[1]**2))

In [ ]:
def pyvista_vector_field_plot(mesh, vector_field, name, factor):
    grid = dolfinx_to_pyvista_mesh(mesh)
    values = np.zeros((mesh.geometry.x.shape[0], 3))
    values[:, :2] = vector_field.compute_point_values()
    grid.point_arrays[name] = values
    grid.set_active_vectors(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    glyphs = grid.glyph(orient=name, factor=factor)
    plotter.add_mesh(glyphs)
    plotter.show()

In [ ]:
pyvista_vector_field_plot(mesh, u_vector, "vector", 5e-2)